In [1]:
!pip install langchain
!pip install langchain_openai
!pip install langgraph
!pip install langchain_community

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import AzureChatOpenAI


os.environ["AZURE_OPENAI_API_KEY"] = 'b3e819600fbe4981be34ef2aa79943e2'

from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def getingPolicyInfo(customerId: str):
    """
    "use this tool when you need to get Policy Information, and you must provide the customer ID"
    """
   
    return f"policy found for {customerId} product: set for life, premium: 20000"

@tool
def getCustomerID(name: str, phone: str):
  """get the customerID base on customer name and phone number, both phone and name are mandatory"""
  return "12345"


tools = [getingPolicyInfo, getCustomerID]
 

llm = AzureChatOpenAI(
    azure_endpoint="https://ik-oai-eastus-2.openai.azure.com/",
    azure_deployment="gpt-4o",
    openai_api_version="2023-09-01-preview",
)


agent_executor = create_react_agent(llm, tools)
prompt = {"messages": [HumanMessage(content=f"What is the premium info for Asa Choi?")]}
for chunk in agent_executor.stream(prompt):
    if(chunk.get("tools")):
      m = chunk["tools"]["messages"][0]
      print(f"{m.name}, {m.content}")
 

getCustomerID, 12345
getingPolicyInfo, policy found for 12345 product: set for life, premium: 20000
